In [ ]:
"""
def load_train_dataframe(id):
    train_data = np.load("train/train/{}.npy".format(id))
    return pd.DataFrame(data=train_data)
def load_test_dataframe(id):
    test_data = np.load("test/test/{}.npy".format(id))
    return pd.DataFrame(data=test_data)

# load training
traindata = []
for id in df_train['Id']:
    dfi = load_train_dataframe(id)
    traindata.append(dfi.values)
print('Done train')
# load test
testdata = []
for id in df_test['Id']:
    dfi = load_test_dataframe(id)
    testdata.append(dfi.values)
print('Done test')

np.save('traindata.npy', traindata)

np.save('testdata.npy', testdata)

"""

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

print('Done')
df_train = pd.read_csv('train_kaggle.csv')
df_test = pd.read_csv('sample_solution.csv')

print('training data: ',df_train.shape,'\ntest data: ', df_test.shape)

Done
training data:  (18662, 2) 
test data:  (6051, 2)


In [2]:
from sklearn.preprocessing import normalize

def process(dfs):
    data = []
    for df in dfs:
        out = np.zeros((df.shape[0],74+1))
        out[:,0:8] = df[:,0:8]
        out[:,8:12] = df[:,8:12]
        out[:,12:28] = df[:,12:28]
        out[:,28:44] = df[:,28:44]
        out[:,44:52] = df[:,44:52]
        out[:,52:64] = df[:,52:64]
    #     out[:,6] = dfs[:,64:80].sum(axis=1) ignore
    #     out[:,7] = dfs[:,80:92].sum(axis=1) ignore
        out[:,64:74] = df[:,92:102]
        out[:,74] = df.shape[0]/100 # Store length of the data
        data.append(out)                 
    return data

def pad_data(dfs):
    padsize = 500
    data = []
    for df in dfs:
        diff = padsize-df.shape[0]
        if diff >0:
            df = np.pad(df, [(diff, 0), (0,0)], 'constant') # I use padding on the front, focus on learning from the back
        else:
            df = df[-padsize:]
        data.append(df)
        
    data = np.stack(data)
    for i in range(data.shape[0]):
        data[i,:,:] = normalize(data[i,:,:],axis=0) # Data normalized (Minimal effect)
    return data

# Load data + Process + Pad
xtrain = np.load('traindata.npy',allow_pickle=True)
xtrain = process(xtrain)
xtrain = pad_data(xtrain)
# xtest = np.load('testdata.npy',allow_pickle=True)
# xtest = process(xtest)

In [21]:
# One hot each string features, then one-hot the overall (Should be done before padding & normalizing the data)
def onehot(input): 
    df = pd.DataFrame(data=input)
    ohot = df.apply(lambda x: pd.factorize(x)[0])
    return np.array(ohot)

# For loop for all examples
APIname = pd.DataFrame(data=xtrain[0,:,:8])

code = onehot(APIname) # onehot by features

for i in range(8):
    APIname_code += 10**i*code[:,i]
APIname_code

array([       0,  1000000, 21100101, 41000001,  1001010, 41000001,
              0,    10100, 41000001,  1001010, 41000001,        0,
        2010012, 41000001,  1001010, 41000001,        0,  2010102,
       41000001,  1001010, 41000001,        0,  1010200,  2010012,
        2010102,  1010200,  2010012,  2010102,  1010200, 41000001,
        1001010, 41000001,        0,  2010012,  2010102,  1010200,
       41000001,  1001010, 41000001,        0,   120000, 41000001,
        1001010, 41000001,        0,  1011120,  1010100,  1010302,
        1010302,  1010302,  1010302,  1010302,  1010302,  1010302,
        1010302,  1010302,  1010302,  1010302,  1010302,  1010302,
        1010302,  1010302,  1010302,  1010302,  1010302,  1010200,
        1010100,  1010100,  1010100,  1010100,  1010302,  1010200,
       23002031,        0,  1010100,  1010100,  1010200,  1010302,
        1010200,  1010100,  1010100,  1010200,  1010302,  1010200,
              0,  2010001,  1010100,  1010100,  1010200,  1010

In [22]:
onehot(APIname_code) 
# This onehot to be shared among all data samples
# Purpose: Certain combination are only shown by malwares
# Need more weight on this part, still figuring

array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 5],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 6],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 7],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 8],
       [ 6],
       [ 7],
       [ 8],
       [ 6],
       [ 7],
       [ 8],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 6],
       [ 7],
       [ 8],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 9],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [10],
       [11],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [ 8],
       [11],
       [11],
       [11],
       [11],
       [12],
       [ 8],
       [13],
       [ 0],
       [11],
       [11],
       [ 8],

In [24]:
# For loop for all examples
APIcat = pd.DataFrame(data=xtrain[0,:,8:12])

code = onehot(APIcat) # onehot by features

APIcat_code = np.zeros_like(code[:,0])
for i in range(4):
    APIcat_code += 10**i*code[:,i]
APIcat_code

array([   0, 1110, 2110,    0,    0,    0,    0, 2110,    0,    0,    0,
          0, 3121,    0,    0,    0,    0, 3121,    0,    0,    0,    0,
       3121, 3121, 3121, 3121, 3121, 3121, 3121,    0,    0,    0,    0,
       3121, 3121, 3121,    0,    0,    0,    0, 3130,    0,    0,    0,
          0, 2100, 3121, 3121, 3121, 3121, 3121, 3121, 3121, 3121, 3121,
       3121, 3121, 3121, 3121, 3121, 3121, 3121, 3121, 3121, 3121, 3121,
       3121, 3121, 3121, 3121, 3121, 3121, 2110,    0, 3121, 3121, 3121,
       3121, 3121, 3121, 3121, 3121, 3121, 3121,    0,    0, 3121, 3121,
       3121, 3121, 3121, 3121,    0,    0, 3121, 3121, 3121, 3121, 3121,
       3121,    0, 3121, 3121, 2110,    0, 2110,    0,    0, 2100, 1201,
       1201, 2110, 2110,    0,    0, 3121, 3121, 3121, 3342, 3121, 3121,
       3121, 3121, 3121, 3121, 3121, 3121, 3121,    0,    0, 3121,    0,
          0, 1201,    0, 1201, 1201, 2110,    0, 1201,    0, 1201,    0,
          0, 3342, 1201, 1201, 3342,    0,    0, 12

In [25]:
onehot(APIname_code) 
# This onehot to be shared among all data samples
# Purpose: Certain combination are only shown by malwares
# Need more weight on this part, still figuring

array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 5],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 6],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 7],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 8],
       [ 6],
       [ 7],
       [ 8],
       [ 6],
       [ 7],
       [ 8],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 6],
       [ 7],
       [ 8],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [ 9],
       [ 3],
       [ 4],
       [ 3],
       [ 0],
       [10],
       [11],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [12],
       [ 8],
       [11],
       [11],
       [11],
       [11],
       [12],
       [ 8],
       [13],
       [ 0],
       [11],
       [11],
       [ 8],

In [5]:
ytrain = np.array(df_train['Label'])
print('Done')

Done


In [6]:
import tensorflow as tf
from tensorflow import keras

inputs = keras.Input(shape=(xtrain.shape[1], xtrain.shape[2])) 
x = keras.layers.Conv1D(filters=6, kernel_size=5, padding='same', activation='relu')(inputs)
x = keras.layers.MaxPooling1D(pool_size=2)(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Conv1D(filters=16, kernel_size=5, padding='same', activation='relu')(x)
x = keras.layers.MaxPooling1D(pool_size=2)(x)
x = keras.layers.Dropout(0.4)(x)
x = keras.layers.Flatten()(x)
x = keras.layers.Dense(120, activation='relu')(x)
x = keras.layers.Dense(84, activation='relu')(x)
x = keras.layers.Dense(2, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=x)

model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

# Take a look at the model summary
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 500, 75)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 6)            2256      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 250, 6)            0         
_________________________________________________________________
dropout (Dropout)            (None, 250, 6)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 250, 16)           496       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 125, 16)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 125, 16)           0     

In [7]:
from tensorflow.keras.utils import to_categorical

model.fit(xtrain, to_categorical(ytrain), epochs=40)

Train on 18662 samples
Epoch 1/40
18662/18662 [==============================] - 18s 985us/sample - loss: 0.3191 - accuracy: 0.8633
Epoch 2/40
18662/18662 [==============================] - 18s 945us/sample - loss: 0.2204 - accuracy: 0.9119
Epoch 3/40
18662/18662 [==============================] - 16s 849us/sample - loss: 0.1978 - accuracy: 0.9227
Epoch 4/40
18662/18662 [==============================] - 16s 839us/sample - loss: 0.1815 - accuracy: 0.9274
Epoch 5/40
18662/18662 [==============================] - 16s 839us/sample - loss: 0.1751 - accuracy: 0.9306
Epoch 6/40
18662/18662 [==============================] - 16s 841us/sample - loss: 0.1617 - accuracy: 0.9366
Epoch 7/40
18662/18662 [==============================] - 16s 880us/sample - loss: 0.1534 - accuracy: 0.9379
Epoch 8/40
18662/18662 [==============================] - 16s 855us/sample - loss: 0.1483 - accuracy: 0.9418
Epoch 9/40
18662/18662 [==============================] - 16s 852us/sample - loss: 0.1448 - accuracy: 0.9

In [8]:
# Load data + Process + Pad
xtest = np.load('testdata.npy',allow_pickle=True)
xtest = process(xtest)
xtest = pad_data(xtest)

ytest = model.predict(xtest)
print('Done')

Done


In [9]:
df_test['Predicted'] = ytest[:, 0]

In [10]:
df_test.to_csv('Normalized_w_DataLength.csv', index=False)